<a href="https://colab.research.google.com/github/waltechel/embedding/blob/master/embeddingChapter04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 해당 소스는 구글 코랩으로 Embedding을 작성한 것입니다. 
# 저자 이기창 님의 소스는 Docker-gpu가 있으면 완벽히 작동합니다만, gpu는 많은 경우 가정에서 구비되어 있지 않고, 
# 그럼에도 불구하고 일부 작동되지 않는 소스에 대해 코랩을 활용하여 실습해 보기 위함입니다.
# 이미지를 띄우는 데 한세월이 걸리는 가정용 도커와 달리, 코랩은 어마어마하게 빠르게 동작합니다.
# 소스를 공유해주신 이기창 님, 이렇게 편리하고 좋은 자원을 나누어주신 Google colabortory 께 감사합니다.

In [1]:
ls

sample_data/


In [2]:
cd ..

/


In [3]:
ls

bin/      dev/   lib32/  notebooks/  run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  opt/        sbin/  tmp/
content/  home/  media/  proc/       srv/   tools/
datalab/  lib/   mnt/    root/       sys/   usr/


In [7]:
mkdir notebooks

In [8]:
ls

bin/      dev/   lib32/  notebooks/  run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  opt/        sbin/  tmp/
content/  home/  media/  proc/       srv/   tools/
datalab/  lib/   mnt/    root/       sys/   usr/


In [4]:
cd notebooks

/notebooks


In [5]:
ls

embedding/


In [10]:
!git clone https://github.com/waltechel/embedding.git

Cloning into 'embedding'...
remote: Enumerating objects: 1749, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1749 (delta 79), reused 67 (delta 33), pack-reused 1618
Receiving objects: 100% (1749/1749), 453.92 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (1164/1164), done.


In [6]:
ls

embedding/


In [7]:
cd embedding/

/notebooks/embedding


In [8]:
ls

data/                                   models/
docker/                                 preprocess/
docs/                                   preprocess.sh*
embeddingChapter03.ipynb                README.md
embeddingChapter04.ipynb                requirements.txt
LICENSE                                 sentmodel.sh*
Miniconda3-py37_4.9.2-Linux-x86_64.sh*  wordmodel.sh*


In [13]:
# 108p 3.3.4. 형태소 분석이 완료된 데이터 다운로드
# 형태소 분석에 시간을 투자하고 싶지 않은 독자들은 다음 소스를 실행하면 형태소 분석을 마친 데이터셋을 한꺼번에 내려받을 수 있다.
!bash preprocess.sh dump-tokenized --yes

download tokenized data...
--2021-08-23 17:56:12--  https://docs.google.com/uc?export=download&confirm=HHkm&id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx
Resolving docs.google.com (docs.google.com)... 142.250.141.139, 142.250.141.138, 142.250.141.100, ...
Connecting to docs.google.com (docs.google.com)|142.250.141.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-c0-docs.googleusercontent.com/docs/securesc/f0sguv9pbf6si38d5qq6gfgvk2bv8s52/rkrcirvf9b17gfep97ehb9sc74891h0a/1629741300000/05170634686643261154/07732646976846605902Z/1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx?e=download [following]
--2021-08-23 17:56:12--  https://doc-10-c0-docs.googleusercontent.com/docs/securesc/f0sguv9pbf6si38d5qq6gfgvk2bv8s52/rkrcirvf9b17gfep97ehb9sc74891h0a/1629741300000/05170634686643261154/07732646976846605902Z/1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx?e=download
Resolving doc-10-c0-docs.googleusercontent.com (doc-10-c0-docs.googleusercontent.com)... 142.250.141.132, 2

# chapter 04 단어 수준 임베딩(111p)
- 4장에서는 다양한 단어 수준 임베딩 모델을 설명한다.
- NPLM, Word2Vec, FastText 등은 예측 기반 모델
- LSA, GloVe, Swivel 등은 행렬 분해 기반의 기법
- 가중 임베딩은 단어 임베딩을 문장 수준으로 확장하는 방법

## 4.1. NPLM
### 4.1.1. 모델 기본 구조
- 기존 언어 모델의 단점
  - 존재하지 않는 n - gram 값을 0으로 부여한다.
  - 문장의 장기 의존성을 포착하기 어렵다(장기 기억을 갖기 어렵다)
  - 단어 문장 간의 유사성을 계산할 수 없다.
- NPLM은 기존 언어 모델의 단점을 일부 해결한 모델
- NPLM : Neural Probablistic Language Model

### 4.1.2. NPLM의 학습
- 단어 시퀀스가 주어졌을 때 다음 단어가 무엇인지 맞추는 과정에서 학습
- 직전까지 등장한 N - 1개의 단어로 다음 단어를 맞추는 것이 n-gram
- NPLM 말단의 출력은 |V| 차원의 스코어 벡터에 소프트맥스 함수를 적용한 |V|차원의 확률 벡터
  - 소프트맥스 함수 : 다중 분류에서 주로 사용하는 함수, 확률의 총합이 1이 되는 함수
- 가장 높은 요소의 인덱스에 해당하는 단어가 실제 정답 단어와 일치하도록 학습
- 스코어 벡터의 계산 공식 y = b + Wx + Utanh(d + Hx)

### 4.1.3. NPLM과 의미 정보
- NPLM은 그 자체로 언어 모델 역할을 수행할 수 있다.
- NPLM은 문장이 말뭉치에 없어도 문맥이 비슷한 문장을 참고해 확률을 부여한다.

## 4.2. Word2Vec
### 4.2.1. 모델 기본 구조
- CBOW : 주변에 있는 문맥 단어를 가지고 타깃 단어를 맞춘다(빈칸 추론과 유사하다)
- skip-gram : 타깃 단어를 가지고 주변 문맥 단어가 무엇인지 예측한다.

### 4.2.2. 학습 데이터 구축
- positive sample : 타깃 단어와 그 주변에 실제로 등장한 문맥 단어
- negative sample : 타깃 단어와 그 주변에 등장하지 않은 단어
- skip-gram 모델은 negative sample이 무한정 발생할 수 있으므로 엄청나게 많은 학습 데이터 쌍을 만들 수 있다. 
- negative sampling : 타깃 단어와 문맥 단어의 쌍이 주어졌을 때 positive, negative를 구분하는 방법

### 4.2.3. 모델 학습



In [14]:
ls

data/                                   models/
docker/                                 preprocess/
docs/                                   preprocess.sh*
embeddingChapter03.ipynb                README.md
embeddingChapter04.ipynb                requirements.txt
LICENSE                                 sentmodel.sh*
Miniconda3-py37_4.9.2-Linux-x86_64.sh*  wordmodel.sh*


In [9]:
cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt

In [10]:
corpus_fname = "/notebooks/embedding/data/tokenized/corpus_mecab.txt"
model_fname = "/notebooks/embedding/data/word-embeddings/word2vec/word2vec"

In [11]:
!pip install -U gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/site-packages (4.0.1)


In [19]:
!which python # should return /usr/local/bin/python

/usr/local/bin/python


In [20]:
!python --version

Python 3.7.11


In [21]:
%env PYTHONPATH=

env: PYTHONPATH=


In [22]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.9.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.9.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-08-23 17:47:57--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90040905 (86M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.9.2-Linux-x86_64.sh’

Miniconda3-py37_4.9 100%[===================>]  85.87M   145MB/s    in 0.6s    

2021-08-23 17:47:58 (145 MB/s) - ‘Miniconda3-py37_4.9.2-Linux-x86_64.sh’ saved [90040905/90040905]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2020.10.14=0
    - certifi==2020.6.20=pyhd3eb1b0_3
    - cffi==1.14.3=py37h261ae71_2
    - chardet==3.0.4=py37h06a4308_1003
    - conda-packa

In [23]:
!which conda  # should return /usr/local/bin/conda

/usr/local/bin/conda


In [24]:
!conda --version

conda 4.9.2


In [25]:
# list the files/directories in the current directory
!ls /usr/local/lib/python3.7/dist-packages

absl
absl_py-0.12.0.dist-info
alabaster
alabaster-0.7.12.dist-info
albumentations
albumentations-0.1.12.dist-info
altair
altair-4.1.0.dist-info
apiclient
appdirs-1.4.4.dist-info
appdirs.py
apt
apt_inst.cpython-37m-x86_64-linux-gnu.so
apt_inst.pyi
apt_pkg.cpython-37m-x86_64-linux-gnu.so
apt_pkg.pyi
aptsources
argcomplete
argcomplete-1.12.3.dist-info
argon2
argon2_cffi-20.1.0.dist-info
arviz
arviz-0.11.2.dist-info
astor
astor-0.8.1.dist-info
astropy
astropy-4.3.1.dist-info
astunparse
astunparse-1.6.3.dist-info
atari_py
atari_py-0.2.9.dist-info
atari_py.libs
atomicwrites
atomicwrites-1.4.0.dist-info
attr
attrs-21.2.0.dist-info
audioread
audioread-2.1.9.dist-info
autograd
autograd-1.3.dist-info
babel
Babel-2.9.1.dist-info
backcall
backcall-0.2.0.dist-info
beautifulsoup4-4.6.3.dist-info
bin
bleach
bleach-4.0.0.dist-info
blis
blis-0.4.1.dist-info
bokeh
bokeh-2.3.3.dist-info
bottleneck
Bottleneck-1.3.2.dist-info
branca
branca-0.4.2.dist-info
bs4
bs4-0.0.1.dist-info
bson
cachecontrol
CacheCont

In [26]:
# requirements.txt 생성
p = """
gensim==3.8.1
pyLDAvis==2.1.2
spacy==2.2.3
scikit-learn==0.23.1
seaborn==0.11.0
squarify==0.4.3
ipykernel
nltk
pandas
scipy
"""

c = """text_file = open("requirements.txt", "w+");text_file.write(p);text_file.close()""" 

exec(c)

In [27]:
# 가상환경 만들기
!conda create -n envname python==3.7

Solving environment: | failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/envs/envname

  added / updated specs:
    - python==3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |            1_gnu          22 KB
    ca-certificates-2021.7.5   |       h06a4308_1         113 KB
    certifi-2021.5.30          |   py37h06a4308_0         139 KB
    libedit-3.1.20210216       |       h27cfd23_1         167 KB
    libffi-3.2.1               |    hf484d3e_1007          48 KB
    libgcc-ng-9.3.0            |      h5101ec6_17         4.8 MB
    libgomp-9.3

In [28]:
# 먼저 가상환경 실행시키고 가상환경 확인
!source activate envname && conda env list

# conda environments:
#
base                     /usr/local
envname               *  /usr/local/envs/envname



In [29]:
# 설치파일이 있다면 설치하기
!conda install --channel conda-forge --file requirements.txt --yes

Solving environment: | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [30]:
!conda install -c anaconda gensim --yes

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.10.14 |                0         128 KB  anaconda
    certifi-2020.6.20          |           py37_0         159 KB  anaconda
    gensim-3.8.0               |   py37h962f231_0        22.7 MB  anaconda
    openssl-1.1.1h             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:        26.8 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2021.5.3~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-

In [ ]:
## 이게 진짜 오래 걸리는 소스
from gensim.models import Word2Vec
corpus = [sent.strip().split(" ") for sent in open(corpus_fname, 'r').readlines()]
model = Word2Vec(corpus, size=100, workers=4, sg=1)
model.save(model_fname)

In [12]:
ls

data/                                   models/
docker/                                 preprocess/
docs/                                   preprocess.sh*
embeddingChapter03.ipynb                README.md
embeddingChapter04.ipynb                requirements.txt
LICENSE                                 sentmodel.sh*
Miniconda3-py37_4.9.2-Linux-x86_64.sh*  wordmodel.sh*


In [13]:
cd /notebooks/embedding/

/notebooks/embedding


In [17]:
pip install -U soynlp

     |████████████████████████████████| 416 kB 8.5 MB/s 
     |████████████████████████████████| 296 kB 53.0 MB/s 


In [18]:
!python models/word_utils.py --method train_word2vec --input_path data/tokenized/corpus_mecab.txt --output_path data/word-embeddings/word2vec

/usr/local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
Traceback (most recent call last):
  File "models/word_utils.py", line 10, in <module>
    from preprocess import get_tokenizer
  File "/notebooks/embedding/preprocess/__init__.py", line 1, in <module>
    from .supervised_nlputils import get_tokenizer, post_processing
  File "/notebooks/embedding/preprocess/supervised_nlputils.py", line 2, in <module>
    from khaiii import KhaiiiApi
ModuleNotFoundError: No module named 'khaiii'


In [22]:
!git clone https://github.com/kakao/khaiii.git

##출처: https://somjang.tistory.com/entry/Ubuntu에서-Khaiii-설치하기 [솜씨좋은장씨]

Cloning into 'khaiii'...
remote: Enumerating objects: 1016, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 1016 (delta 46), reused 76 (delta 29), pack-reused 877
Receiving objects: 100% (1016/1016), 33.06 MiB | 27.00 MiB/s, done.
Resolving deltas: 100% (404/404), done.


In [23]:
cd khaiii

/notebooks/embedding/khaiii


In [24]:
mkdir build 

In [25]:
cd build

/notebooks/embedding/khaiii/build


In [27]:
!cmake ..

-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test fma_compiles
-- Performing Test fma_compiles - Success
-- Performing Test fma_runs
-- Performing Test fma_runs - Success
-- [khaiii] fused multiply add option enab

In [29]:
!make all

Scanning dependencies of target obj_khaiii
[  3%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Config.cpp.o
[  7%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Embed.cpp.o
[ 11%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/ErrPatch.cpp.o
[ 15%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/KhaiiiImpl.cpp.o
[ 19%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Morph.cpp.o
[ 23%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Preanal.cpp.o
[ 26%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Resource.cpp.o
[ 30%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Restore.cpp.o
[ 34%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Sentence.cpp.o
[ 38%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Tagger.cpp.o
[ 42%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Trie.cpp.o
[ 4

In [30]:
!make resource

Scanning dependencies of target resource
INFO:root:config: {
    "cutoff": 1,
    "embed_dim": 35,
    "hidden_dim": 320,
    "model_id": "munjong.cut1.win4.sdo0.1.emb35.lr0.001.lrd0.9.bs500",
    "rsc_src": "../rsc/src",
    "window": 4
}
INFO:root:vocab.in: 5621 entries, 0 cutoff
INFO:root:vocab.out: 500 entries, 0 cutoff
INFO:root:restore.dic: 4303 entries
INFO:root:preanal.auto
INFO:root:preanal.manual
INFO:root:trie saved: /notebooks/embedding/khaiii/build/share/khaiii/preanal.tri
INFO:root:total nodes: 120812
INFO:root:expected size: 1932992
INFO:root:value saved: /notebooks/embedding/khaiii/build/share/khaiii/preanal.val
INFO:root:total entries: 83373
INFO:root:expected size: 593730
INFO:root:restore.key: 17212
INFO:root:restore.val: 68848
INFO:root:restore.one: 503
INFO:root:base.errpatch.auto
INFO:root:base.errpatch.manual
INFO:root:trie saved: /notebooks/embedding/khaiii/build/share/khaiii/errpatch.tri
INFO:root:total nodes: 2067
INFO:root:expected size: 33072
INFO:root:lengt

In [32]:
cd ..

/notebooks/embedding/khaiii


In [33]:
ls

build/          CONTRIBUTING.md  LICENSE    README.md         src/
cmake/          docker/          munjong/   requirements.txt  train/
CMakeLists.txt  include/         NOTICE.md  rsc/


In [34]:
cd ..

/notebooks/embedding


In [37]:
!python models/word_utils.py --method train_word2vec --input_path data/tokenized/corpus_mecab.txt --output_path data/word-embeddings/word2vec

Traceback (most recent call last):
  File "models/word_utils.py", line 10, in <module>
    from preprocess import get_tokenizer
  File "/notebooks/embedding/preprocess/__init__.py", line 1, in <module>
    from .supervised_nlputils import get_tokenizer, post_processing
  File "/notebooks/embedding/preprocess/supervised_nlputils.py", line 2, in <module>
    from khaiii import KhaiiiApi
ImportError: cannot import name 'KhaiiiApi' from 'khaiii' (unknown location)


In [36]:
pip install python-Levenshtein

     |████████████████████████████████| 50 kB 4.4 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=171558 sha256=93ddb156ca7c46ff15d400ac5f2cb88c7276937308aa6a07f3ff58b4cc4416dc
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
## 코드 4-5 코사인 유사도 상위 단어 목록 체크 코드와 그 예시
from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator("/notebooks/embedding/data/word-embeddings/word2vec/word2vec", method="word2vec", dim=100, tokenizer_name = 'mecab')
model.most_similar("희망", topn=10)